# 環境設定

In [25]:
import os
import sys

def _is_in_kaggle() -> bool:
  """Whether the current environment is in `Kaggle`."""
  return str(_dh[0]) == '/kaggle/working'


def _is_in_colab() -> bool:
  """Whether the current environment is in `Colab`."""
  return 'google.colab' in str(get_ipython())

In [26]:
if _is_in_kaggle():
  print('in kaggle')
elif _is_in_colab():
  print('in colab')
  from google.colab import drive
  drive.mount('/content/drive')  # mount google drive
  sys.path.append('/content/drive/MyDrive/Colab Notebooks/my-modules')  # path from drive
else:
  print('in local')

in colab
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## pipeline
transformersの基本的な機能としてpipelineがある。
これは指定のモデルに対して、下記の一連の操作を自動実行してくれる機能である。
1. 入力を前処理
2. モデルに入力
3. 出力を後処理

In [28]:
from transformers import pipeline, AutoTokenizer, AutoModel, AutoModelForSequenceClassification
import torch
from IPython.display import display

In [29]:
classifier = pipeline("sentiment-analysis")
sentences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
    "I do not dislike this.",
]
classifier(sentences)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9598046541213989},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455},
 {'label': 'POSITIVE', 'score': 0.9790695309638977}]

pipelineで自動実行される一連の処理に対して、それぞれtransformersでは個別にも機能が実装されており、AutoTokenizerとAutoModelがそれらに該当する。


## AutoTokenizer

まず、前処理については、AutoTokenizerを使う。
AutoTokenizerは、入力の前処理を自動的に行い、入力をトークンに分割し、それらをIDに変換する。
この処理はモデルごとに行われるため、AutoTokenizerではfrom_pretrainedでモデルを指定する必要がある。

In [30]:
# モデルの指定とTokenizerの読み込み
ckpt = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(ckpt)

raw_inputs = [
    "Odapai is a man who has big tits.",
    "Satio made us very happy.",
    "We worked hard everyday at the dirty factory.",
]

# トークナイズ
inputs = tokenizer(
    raw_inputs,
    padding=True,
    truncation=True,
    return_tensors="pt"
    )

display(inputs)

{'input_ids': tensor([[  101,  1051,  2850,  4502,  2072,  2003,  1037,  2158,  2040,  2038,
          1037,  2502, 25671,  1012,   102],
        [  101,  2938,  3695,  2081,  2149,  2200,  3407,  1012,   102,     0,
             0,     0,     0,     0,     0],
        [  101,  2057,  2499,  2524, 10126,  2012,  1996,  6530,  4713,  1012,
           102,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])}

## AutoModel

モデルの読み込みとモデルへの入力については、AutoModelを使う。
このときの出力は、AutoModelの最終層の出力と同様の形式になり、下記の例ではhidden_size: 768であり、この最終層出力が入力文章の分散表現となる。

In [31]:
# モデルの指定とAutoModelの読み込み
model = AutoModel.from_pretrained(ckpt)  # AutoTokenizer と AutoModel は同様のモデルを指定する

# モデルに入力
outputs = model(**inputs)

# 出力
## last_hidden_state: 最終層の出力
## サイズが大きいのでshapeのみ確認する([batch_size, sequence_length, hidden_size])
display(f'outputs.last_hidden_state.shape: {outputs.last_hidden_state.shape}')

'outputs.last_hidden_state.shape: torch.Size([3, 15, 768])'

このモデルの最終層出力は、各タスクに対応するような追加の層を与える事で個別のタスクを解くことができるようになる。
そのための機能の一つが、AutoModelForSequenceClassificationである。
AutoModelでは、入力文章の分散表現の獲得までを行なったが、AutoModelForSequenceClassificationは分散表現からラベルを予測するまでを一貫して行う。

In [32]:
# AutoModelForSequenceClassificationの読み込み
model = AutoModelForSequenceClassification.from_pretrained(ckpt)

# モデルに入力
outputs = model(**inputs)

# 出力
print(f'outputs.logits: {outputs.logits}')

outputs.logits: tensor([[-0.6493,  0.7493],
        [-4.3593,  4.6947],
        [ 2.8191, -2.5292]], grad_fn=<AddmmBackward0>)


生の出力(logits)では解釈ができないため、logitsをSoftmaxで確率に変換する。

In [34]:
import torch.nn.functional as F

# 推論
probs = F.softmax(outputs.logits, dim=-1)
print(f'probs: {probs}')

probs: tensor([[1.9802e-01, 8.0198e-01],
        [1.1692e-04, 9.9988e-01],
        [9.9527e-01, 4.7341e-03]], grad_fn=<SoftmaxBackward0>)
{0: 'NEGATIVE', 1: 'POSITIVE'}
